<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/vacuum_stabilization_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# ------------------------------------------------------------------------------
# 1. Model Definition: Multi-task MLP with physics-informed branch
# ------------------------------------------------------------------------------
class VacuumAI(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_classes):
        super(VacuumAI, self).__init__()
        # Shared trunk
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.ln1 = nn.LayerNorm(hidden_dim)
        self.act = nn.GELU()
        self.dropout = nn.Dropout(0.2)

        # Classification head
        self.cls_fc = nn.Linear(hidden_dim, n_classes)

        # Regression head for vacuum energy
        self.reg_fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        h = self.act(self.ln1(self.fc1(x)))
        h = self.dropout(h)

        logits = self.cls_fc(h)
        logp   = F.log_softmax(logits, dim=-1)

        energy = self.reg_fc(h)  # continuous output

        return logp, energy

# ------------------------------------------------------------------------------
# 2. Synthetic Dataset (Placeholder – replace with real data & physics model)
# ------------------------------------------------------------------------------
class SyntheticVacuumDataset(Dataset):
    def __init__(self, N=10000):
        # 4 fluctuation variables ~ N(0,1)
        X = np.random.randn(N, 4).astype(np.float32)

        # Define stability class by sum of features (toy rule)
        sums = X.sum(axis=1)
        y_cls = np.zeros(N, dtype=np.int64)
        y_cls[sums > -0.5] = 1
        y_cls[sums >  0.5] = 2

        # Physics-based vacuum energy (toy model: sum of squares)
        y_energy = (X**2).sum(axis=1, keepdims=True).astype(np.float32)

        self.X = torch.from_numpy(X)
        self.y_cls = torch.from_numpy(y_cls)
        self.y_energy = torch.from_numpy(y_energy)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y_cls[idx], self.y_energy[idx]

# ------------------------------------------------------------------------------
# 3. DataLoader
# ------------------------------------------------------------------------------
batch_size = 128
dataset = SyntheticVacuumDataset(N=12000)
train_size = int(0.8 * len(dataset))
val_size   = len(dataset) - train_size
train_ds, val_ds = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=batch_size)

# ------------------------------------------------------------------------------
# 4. Instantiate Model, Losses, Optimizer
# ------------------------------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model     = VacuumAI(input_dim=4, hidden_dim=32, n_classes=3).to(device)
cls_loss  = nn.NLLLoss()
reg_loss  = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

# ------------------------------------------------------------------------------
# 5. Simple Physics-Informed Regularizer
#    Penalize negative energy predictions
# ------------------------------------------------------------------------------
def physics_penalty(energy_pred):
    # enforces vacuum energy >= 0
    return torch.mean(F.relu(-energy_pred))

# ------------------------------------------------------------------------------
# 6. Training Loop
# ------------------------------------------------------------------------------
n_epochs = 30
history = {'train_loss':[], 'val_loss':[], 'val_acc':[]}

for epoch in range(1, n_epochs+1):
    model.train()
    running_loss = 0.0

    for Xb, yb_cls, yb_eng in train_loader:
        Xb, yb_cls, yb_eng = Xb.to(device), yb_cls.to(device), yb_eng.to(device)

        optimizer.zero_grad()
        logp, eng_pred = model(Xb)

        loss_cls = cls_loss(logp, yb_cls)
        loss_reg = reg_loss(eng_pred, yb_eng)
        loss_phys = physics_penalty(eng_pred)

        # Weighted sum of tasks + physics penalty
        loss = loss_cls + 0.5 * loss_reg + 0.1 * loss_phys
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * Xb.size(0)

    train_loss = running_loss / len(train_loader.dataset)
    history['train_loss'].append(train_loss)

    # Validation
    model.eval()
    val_loss = 0.0
    correct  = 0
    total    = 0

    with torch.no_grad():
        for Xb, yb_cls, yb_eng in val_loader:
            Xb, yb_cls, yb_eng = Xb.to(device), yb_cls.to(device), yb_eng.to(device)
            logp, eng_pred = model(Xb)

            lc = cls_loss(logp, yb_cls)
            lr = reg_loss(eng_pred, yb_eng)
            lp = physics_penalty(eng_pred)
            loss = lc + 0.5*lr + 0.1*lp

            val_loss += loss.item() * Xb.size(0)

            preds = logp.argmax(dim=1)
            correct += (preds == yb_cls).sum().item()
            total   += Xb.size(0)

    val_loss /= len(val_loader.dataset)
    val_acc  = correct / total
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)

    print(f"Epoch {epoch:02d}  Train Loss: {train_loss:.4f}  "
          f"Val Loss: {val_loss:.4f}  Val Acc: {val_acc:.3f}")

# ------------------------------------------------------------------------------
# 7. Evaluation & Visualization
# ------------------------------------------------------------------------------
# Confusion Matrix on Validation Set
all_preds, all_true = [], []
model.eval()
with torch.no_grad():
    for Xb, yb_cls, _ in val_loader:
        Xb = Xb.to(device)
        logp, _ = model(Xb)
        preds = logp.argmax(dim=1).cpu().numpy()
        all_preds.append(preds)
        all_true.append(yb_cls.numpy())

y_pred = np.concatenate(all_preds)
y_true = np.concatenate(all_true)

cm = confusion_matrix(y_true, y_pred, labels=[0,1,2])
disp = ConfusionMatrixDisplay(cm, display_labels=["Unstable","Meta","Stable"])
disp.plot(cmap="Blues")
plt.title("Validation Confusion Matrix")
plt.show()

# Loss & Accuracy Curves
plt.figure(figsize=(8,4))
plt.plot(history['train_loss'], label="Train Loss")
plt.plot(history['val_loss'],   label="Val Loss")
plt.plot(history['val_acc'],    label="Val Acc")
plt.legend()
plt.xlabel("Epoch")
plt.tight_layout()
plt.show()